In [1]:
from googleapiclient.discovery import build 
import os
import pandas as pd
from IPython.display import JSON

In [2]:
# reading api_key from file
api_key = ""
with open("key.txt","r") as f:
    api_key = f.read()

In [3]:
    # Get credentials and create an API client

api_service_name = "youtube"
api_version = "v3"
youtube = build (
    api_service_name, api_version, developerKey=api_key )
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"

In [46]:
# -*- coding: utf-8 -*-
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"
def get_channels(youtube,channel_id):
    total_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
        data = {
                "Channel_Name" : item['snippet']['title'],
                "Suscribers" : item['statistics']['subscriberCount'],
                "View" : item['statistics']['viewCount'],
                "TotalVideo" : item['statistics']['videoCount'],
                "PlaylistId" : item['contentDetails']['relatedPlaylists']['uploads']
               }

    total_data.append(data)
    return pd.DataFrame(total_data)

def get_videos(youtube, video_id):
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=video_id
    )
    response = request.execute()
    return JSON(response)

channel_data = get_channels(youtube, channel_id)
video_id = str(channel_data['PlaylistId'][0]).strip()
print(video_id)

get_videos(youtube, video_id)


UUX6OQ3DkcsbYNE6H8uQQuVA


<IPython.core.display.JSON object>